# pip install

In [ ]:
#!pip install lightgbm scikit-learn xgboost

# Imports

In [1]:
import pandas as pd
import numpy as np

from itertools import product

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_log_error

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from pathlib import Path

# Data preparation
We will check for missing data, numerically encode categorical variables, and do some feature engineering in regards to potential oil price lags.

In [2]:
# Notebook location
NOTEBOOK_DIR = Path().resolve()
# Data directory
DATA_DIR = NOTEBOOK_DIR / "data"

#parse_dates=['date'] converts date values into datetime objects
train = pd.read_csv(DATA_DIR / "train.csv", parse_dates=['date'])
test = pd.read_csv(DATA_DIR / 'test.csv', parse_dates=['date'])
holidays_events = pd.read_csv(DATA_DIR / 'holidays_events.csv', parse_dates=['date'])
oil = pd.read_csv(DATA_DIR / 'oil.csv', parse_dates=['date'])
stores = pd.read_csv(DATA_DIR / 'stores.csv')
transactions = pd.read_csv(DATA_DIR / 'transactions.csv', parse_dates=['date'])
#data = pd.read_csv(DATA_DIR / 'data.csv', parse_dates=['date'])

### Check if there are missing dates in the train data

In [3]:
# Get the date range from earliest to latest in train data
date_range = pd.date_range(start=train['date'].min(), end=train['date'].max(), freq='D')

# Get unique dates in train data
train_dates = train['date'].unique()

# Find missing dates
missing_dates = date_range.difference(pd.DatetimeIndex(train_dates))

print(f"Total dates in range: {len(date_range)}")
print(f"Unique dates in train: {len(train_dates)}")
print(f"Missing dates: {len(missing_dates)}")
print(f"\nMissing dates:\n{missing_dates}")


Total dates in range: 1688
Unique dates in train: 1684
Missing dates: 4

Missing dates:
DatetimeIndex(['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'], dtype='datetime64[ns]', freq=None)


We have 4 missing dates in total. This will cause problems in modelling, since timeseries models require a continuous flow time. Forecasting models require need the timeseries input to be complete, not missing any dates.  The following code will fix this and set the target value of sales as 0 on these days.

In [4]:
# Create a complete date range from train data
date_range = pd.date_range(start=train['date'].min(), end=train['date'].max(), freq='D')

# Get all unique combinations of store number and product family
stores_list = train['store_nbr'].unique()
unique_product_families = train['family'].unique()

# Create a MultiIndex with all combinations of date, store_nbr, and family
complete_index = pd.MultiIndex.from_tuples(
    product(date_range, stores_list, unique_product_families),
    names=['date', 'store_nbr', 'family']
)

# Create a complete dataframe
complete_df = pd.DataFrame(index=complete_index).reset_index()

# Merge with original train data
train_complete = complete_df.merge(
    train,
    on=['date', 'store_nbr', 'family'],
    how='left'
)

# Fill missing values i.e. sales and onpromotion with 0
train_complete['sales'] = train_complete['sales'].fillna(0)
train_complete['onpromotion'] = train_complete['onpromotion'].fillna(0)

# 4 missing dates, 54 stores, 33 product families i.e. 4 * 54 * 33 = 7128
print(f"Added records: {len(train_complete) - len(train)}")

train = train_complete

Added records: 7128


### Check missing oil dates

In [5]:
# Get the date range from earliest to latest in train and test data
date_range = pd.date_range(start=train['date'].min(), end=test['date'].max(), freq='D')

# Get unique dates in oil data
oil_dates = oil['date'].unique()

# Find missing dates
missing_dates = date_range.difference(pd.DatetimeIndex(oil_dates))

print(f"Total dates in range: {len(date_range)}")
print(f"Unique dates in oil: {len(oil_dates)}")
print(f"Missing dates: {len(missing_dates)}")

Total dates in range: 1704
Unique dates in oil: 1218
Missing dates: 486


We are once again missing dates, which disrupts our continuous timeseries. We will use the same method to fix it.

In [6]:
# Create complete date range for the data from train and test data
date_range = pd.date_range(start=train['date'].min(), end=test['date'].max(), freq='D')

# Create complete dataframe with all dates
oil_complete = pd.DataFrame({'date': date_range})

# Merge with original oil data
oil_complete = oil_complete.merge(
    oil, 
    on='date', 
    how='left'
)

# Forward fill (use last known price for missing dates)
oil_complete['dcoilwtico'] = oil_complete['dcoilwtico'].ffill()

# Backward fill for any remaining NaNs at the start
oil_complete['dcoilwtico'] = oil_complete['dcoilwtico'].bfill()

# Update oil dataframe
print(f"Number of records after filling missing oil data: {len(oil_complete)}")
print(f"Added records: {len(oil_complete) - len(oil)}")

oil = oil_complete

Number of records after filling missing oil data: 1704
Added records: 486


### Check transactions

In [7]:
# Total transactions count
total_transactions_count = len(train.groupby(["date", "store_nbr"])['sales'].sum())

print(f"Missing records from transactions: {total_transactions_count - len(transactions)}")


store_sales = train.groupby(['date', 'store_nbr'])['sales'].sum().reset_index()

# Merge transactions with sales
transactions_complete = transactions.merge(
    store_sales,
    on=['date', 'store_nbr'],
    how='outer'
).sort_values(by=['date', 'store_nbr'], ignore_index=True)

# For dates where sales are 0, set transactions to 0
transactions_complete.loc[transactions_complete["sales"].eq(0), "transactions"] = 0

# Interpolate missing transactions for each store
transactions_complete['transactions'] = transactions_complete.groupby('store_nbr')['transactions'].transform(
    lambda x: x.interpolate(method='linear')
)

# Round transactions to remove fractions from interpolation
transactions_complete['transactions'] = transactions_complete['transactions'].round().astype(int)

# Drop sales column
transactions_complete = transactions_complete.drop('sales', axis=1)

print(f"Added transactions: {len(transactions_complete) - len(transactions)}")

transactions = transactions_complete

Missing records from transactions: 7664
Added transactions: 7664


### Holidays

We want to create a 0/1 flag variable for holidays in our time series. This way we can account for extra store traffic during holidays quite simply.

In [22]:
# Clean holiday event descriptions and remove "Transfer" holidays, since they are actually not holidays.
holidays_events["description"] = holidays_events.apply(
    lambda x: x["description"].strip().lower().replace(x["locale_name"].lower(), ""), axis=1
).apply(
    #If the holiday is connected to football, we will just categorize it as futbol
    lambda x: "futbol" if "futbol" in x else x
).replace(
    #Removing unneeded words in the names of holidays
    r'\b(de|del|puente|traslado|recupero)\b', '', regex=True
).replace(
    #Removing +1, -2 etc type holidays since these are not actual holidays
    r'[+-]\d+', '', regex=True
).str.strip()
#Transfers are not actually holidays, they have been moved to a different day.
holidays_events = holidays_events[holidays_events["type"] != "Transfer"]

#Build 3 holiday tables for national, regional and local

h = holidays_events[['date', 'locale', 'locale_name']].copy()

# NATIONAL
#We only need to keep the date, since nationals take place all over the country
national = h[h['locale'] == 'National'][['date']].drop_duplicates()
national['national_holiday'] = 1

# REGIONAL → matches on state
#we have to keep the date and the region
regional = h[h['locale'] == 'Regional'][['date', 'locale_name']].drop_duplicates()
regional = regional.rename(columns={'locale_name': 'state'})
regional['regional_holiday'] = 1

# LOCAL → matches on city
#We have to keep the city and date
local = h[h['locale'] == 'Local'][['date', 'locale_name']].drop_duplicates()
local = local.rename(columns={'locale_name': 'city'})
local['local_holiday'] = 1

# --- Merge into data ---
data = pd.concat(
    [train, test], axis=0, ignore_index=True
).merge(
    transactions, on=['date', 'store_nbr'], how='left'
).merge(
    oil, on='date', how='left'
).merge(
    stores, on='store_nbr', how='left'
)
# Merge NATIONAL
data = data.merge(national, on='date', how='left')

# Merge REGIONAL
data = data.merge(regional, on=['date', 'state'], how='left')

# Merge LOCAL
data = data.merge(local, on=['date', 'city'], how='left')

# Final combined holiday flag column
data['holiday'] = (
    data[['national_holiday', 'regional_holiday', 'local_holiday']]
    .fillna(0)
    .max(axis=1)
    .astype(int) #should maybe make into factor
)

# Cleanup
data = data.drop(columns=['national_holiday', 'regional_holiday', 'local_holiday'])

data['holiday'].value_counts()

holiday
0    2774310
1     262218
Name: count, dtype: int64

### Creating one and two day lags for dcoilwtico

In [23]:
# Sort by store_nbr, family, and date to ensure correct order within each group
data = data.sort_values(['store_nbr', 'family', 'date'])

# Create lag-1 column within each store-family combination
data['lag-1-dcoilwtico'] = data.groupby(['store_nbr', 'family'])['dcoilwtico'].shift(1)

# Create lag-2 column (2 days ago)
data['lag-2-dcoilwtico'] = data.groupby(['store_nbr', 'family'])['dcoilwtico'].shift(2)

# Backfill NaNs at the start of each group
data['lag-1-dcoilwtico'] = data.groupby(['store_nbr', 'family'])['lag-1-dcoilwtico'].bfill()
data['lag-2-dcoilwtico'] = data.groupby(['store_nbr', 'family'])['lag-2-dcoilwtico'].bfill()
data = data.sort_values('date')

### Extract date features

In [24]:
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['day_of_week'] = data['date'].dt.dayofweek
data['day_of_year'] = data['date'].dt.dayofyear
data['week_of_year'] = data['date'].dt.isocalendar().week
data['is_weekend'] = (data['date'].dt.dayofweek >= 5).astype(int)

### Label Encoding for categorical features

Since most ML models cannot work with categorical features like "AUTOMOTIVE" etc, we have to numerically encode them for the model to work.

In [25]:
# Get all unique values for each categorical column from data
all_product_families = data['family'].unique()
all_cities = data['city'].unique()
all_states = data['state'].unique()
all_types = data['type'].unique()

# Initialize label encoders
le_product_family = LabelEncoder()
le_city = LabelEncoder()
le_state = LabelEncoder()
le_type = LabelEncoder()

# Fit on all unique values
le_product_family.fit(all_product_families)
le_city.fit(all_cities)
le_state.fit(all_states)
le_type.fit(all_types)

# Transform data dataframe
data['family'] = le_product_family.transform(data['family'])
data['city'] = le_city.transform(data['city'])
data['state'] = le_state.transform(data['state'])
data['type'] = le_type.transform(data['type'])

print("Label encoding complete!")
print(f"Family: {len(all_product_families)} categories")
print(f"City: {len(all_cities)} categories")
print(f"State: {len(all_states)} categories")
print(f"Type: {len(all_types)} categories")

Label encoding complete!
Family: 33 categories
City: 22 categories
State: 16 categories
Type: 5 categories


### Save final version of 'data' dataframe 

In [26]:
data.to_csv('./data/data.csv', index=False)

# Model training

### Splitting test and train data

In [27]:
# Split based on date
train = data[data['date'] < '2017-08-16'].copy()
test = data[data['date'] >= '2017-08-16'].copy()

# Save test IDs before dropping
test_ids = test['id'].copy()

# Drop transactions column from both
train = train.drop('transactions', axis=1)
test = test.drop('transactions', axis=1)

# Drop date column since we have date features
train = train.drop('date', axis=1)
test = test.drop('date', axis=1)

# Drop id column
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

# Drop sales column only from test
test = test.drop('sales', axis=1)

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"Test IDs saved: {len(test_ids)}")

Train shape: (3008016, 19)
Test shape: (28512, 18)
Test IDs saved: 28512


In [28]:
# Separate features and target
X_train = train.drop('sales', axis=1)
y_train = train['sales']

X_test = test 

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Target: {y_train.shape}")

Training set: (3008016, 18)
Test set: (28512, 18)
Target: (3008016,)


### Random Forest model

In [29]:
def train_random_forest_model(X_train, y_train):
    # Initialize Random Forest model
    rf_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=20,
        min_samples_split=10,
        min_samples_leaf=4,
        random_state=42,
        n_jobs=-1,
        verbose=1
    )

    # Train the model
    print("Training Random Forest model...")
    rf_model.fit(X_train, y_train)

    # Make predictions on training set to evaluate
    y_train_pred_rf = np.maximum(0, rf_model.predict(X_train))

    # Calculate metrics
    rmsle_score = np.sqrt(mean_squared_log_error(y_train, y_train_pred_rf))

    return rf_model, rmsle_score

### LightGBM Model

In [30]:
def train_lightgbm_model(X_train, y_train):
    # Initialize LightGBM model
    lgb_model = LGBMRegressor(
        n_estimators=100,
        max_depth=20,
        learning_rate=0.1,
        num_leaves=31,
        min_child_samples=20,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )

    # Train the model
    print("Training LightGBM model...")
    lgb_model.fit(X_train, y_train)

    # Make predictions on training set to evaluate
    y_train_pred_lgb = np.maximum(0, lgb_model.predict(X_train))

    # Calculate metrics
    rmsle_score_lgb = np.sqrt(mean_squared_log_error(y_train, y_train_pred_lgb))

    return lgb_model, rmsle_score_lgb

### XGBoost Model

In [31]:
def train_xgboost_model(X_train, y_train):
    # Initialize XGBoost model
    xgb_model = XGBRegressor(
        n_estimators=100,
        max_depth=10,
        learning_rate=0.1,
        min_child_weight=3,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbosity=1
    )

    # Train the model
    print("Training XGBoost model...")
    xgb_model.fit(X_train, y_train)

    # Make predictions on training set to evaluate
    y_train_pred_xgb = np.maximum(0, xgb_model.predict(X_train))

    # Calculate metrics
    rmsle_score_xgb = np.sqrt(mean_squared_log_error(y_train, y_train_pred_xgb))

    return xgb_model, rmsle_score_xgb

### Create submission

In [32]:
def create_submission(predictions, test_ids, filename='submission.csv'):
    submission = pd.DataFrame({
        'id': test_ids.values.astype(int),
        'sales': predictions
    })
    
    submission.to_csv(filename, index=False)
    return submission

### Model Comparison

In [33]:
xgb_model, xgb_rmsle_score = train_xgboost_model(X_train, y_train)
rf_model, rf_rmsle_score = train_random_forest_model(X_train, y_train)
lgbm_model, lgbm_rmsle_score = train_lightgbm_model(X_train, y_train)

Training XGBoost model...
Training Random Forest model...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  7.3min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    4.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:   11.0s finished


Training LightGBM model...


In [34]:
# Compare all models including ensemble
comparison_df = pd.DataFrame({
    'Model': ['Random Forest', 'LightGBM', 'XGBoost'],
    'RMSLE': [rf_rmsle_score, lgbm_rmsle_score, xgb_rmsle_score]
})

print("\nModel Performance Comparison:")
print(comparison_df.to_string(index=False))
print(f"\nBest model by RMSLE: {comparison_df.loc[comparison_df['RMSLE'].idxmin(), 'Model']}")


Model Performance Comparison:
        Model    RMSLE
Random Forest 0.625781
     LightGBM 2.154993
      XGBoost 1.505900

Best model by RMSLE: Random Forest


### Create submissions

In [ ]:
xgb_y_pred = xgb_model.predict(X_test)
rf_y_pred = rf_model.predict(X_test)
lgbm_y_pred = lgbm_model.predict(X_test)

# Create submission files using the saved test IDs
xgb_submission = create_submission(xgb_y_pred, test_ids, 'xgb_submission.csv')
rf_submission = create_submission(rf_y_pred, test_ids, 'rf_submission.csv')
lgbm_submission = create_submission(lgbm_y_pred, test_ids, 'lgbm_submission.csv')

print("All submission files created successfully!")

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


All submission files created successfully!
